In [14]:
import re
import geopandas as gpd
from shapely.geometry import Polygon
from PyPDF2 import PdfReader

In [15]:
# Função para extrair coordenadas UTM do PDF
def extract_coordinates_from_pdf(pdf_file):
    # Inicializando o leitor de PDF com PdfReader
    pdf_reader = PdfReader(pdf_file)
    coordinates = []
    
    # Expressão regular para coordenadas UTM (formato: N 7.747.897,01 m e E 684.187,64 m)
    utm_pattern = r'N (\d{1,3}(?:\.\d{3})*(?:,\d+)?) m\. e E (\d{1,3}(?:\.\d{3})*(?:,\d+)?) m'
    
    for page_num in range(len(pdf_reader.pages)):
        page = pdf_reader.pages[page_num]
        text = page.extract_text()
        
        # Procurar por padrões de coordenadas UTM
        matches = re.findall(utm_pattern, text)
        
        for match in matches:
            northing = float(match[0].replace('.', '').replace(',', '.'))  # Conversão do formato UTM
            easting = float(match[1].replace('.', '').replace(',', '.'))
            coordinates.append((easting, northing))
    
    return coordinates

In [16]:
# Função para criar um polígono a partir das coordenadas
def create_polygon_from_coordinates(coordinates):
    if len(coordinates) < 3:
        raise ValueError("Pelo menos três coordenadas são necessárias para formar um polígono.")
    
    # Criar um polígono usando as coordenadas
    polygon = Polygon(coordinates)
    return polygon

In [30]:
# Função para salvar o polígono em um GeoDataFrame e exportá-lo para um Geodatabase
def save_polygon_to_geodatabase(polygon, gdb_path, layer_name):
    # Criar um GeoDataFrame
    gdf = gpd.GeoDataFrame(index=[0], crs="EPSG:31983", geometry=[polygon])  # CRS UTM zona 23S como exemplo
    
    # Salvar o GeoDataFrame no Geodatabase especificado
    gdf.to_file(gdb_path, layer=layer_name, driver="GPKG")  # Usando Geopackage como exemplo de geodatabase

In [31]:
# Exemplo de uso do script
pdf_file_path = 'C:/caminho/PDF
gdb_path = 'C:/caminho/geopackage.gpkg'  # Defina o caminho para o geodatabase (GeoPackage)
layer_name = 'nome_camada'

In [32]:
# Extrair coordenadas do PDF
coordinates = extract_coordinates_from_pdf(pdf_file_path)

# Criar um polígono
polygon = create_polygon_from_coordinates(coordinates)

# Salvar o polígono no Geodatabase
save_polygon_to_geodatabase(polygon, gdb_path, layer_name)

print(f"Polígono salvo no geodatabase em {gdb_path}, camada '{layer_name}'")

Polígono salvo no geodatabase em C:/Users/silas.nascimento/OneDrive - Fundação Renova/Documentos/RENOVA/FUNDIARIO/3000253952/Estruturaca_dados_fundiario/poligonais_dados_fundiario.gpkg, camada 'CPLeandroMemorialDescritivoRev01Com_sirgas'
